## **1.0. Data Pre-Processing**

### 1.1. Imports

In [1]:
!pip install transformers datasets scikit-learn torch

In [2]:
import pandas as pd
import random,time,csv
import numpy as np
import math, copy, os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import wandb
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from kaggle_secrets import UserSecretsClient
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE

In [3]:
bank_df = pd.read_csv('/kaggle/input/nlp-project-data-raw/data/bank.csv')
bank_df['Probability'] = np.where(bank_df['Probability'] == 'yes', 1, 0)
mean = bank_df.loc[:,"age"].mean()
bank_df['age'] = np.where(bank_df['age'] >= 30, 1, 0)

In [4]:
# A helper function to split data into train and test sets
def split_data(df, target, test_size=0.3, random_state=42, stratify=True):
    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in DataFrame.")
    X = df.drop(columns=[target])
    y = df[target]
    if stratify:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )
    return X_train, X_test, y_train, y_test

# Bank Marketing Dataset
X_train_bank, X_test_bank, y_train_bank, y_test_bank = split_data(bank_df, target="Probability")

In [5]:
# Apply SMOTE
smote = SMOTE(random_state=42)

In [6]:
# Encoding the X_train and X_test to prepare Bank Dataset for SMOTE
X_train_bank_encoded = pd.get_dummies(X_train_bank, drop_first=True)
X_test_bank_encoded = pd.get_dummies(X_test_bank, drop_first=True)

In [7]:
# Bank Marketing Dataset with SMOTE
X_train_bank_smote, y_train_bank_smote = smote.fit_resample(X_train_bank_encoded, y_train_bank)

In [8]:
# -------------------------------------
# 1. Data Preparation
# -------------------------------------

# We will combine X and y back into a single DataFrame for convenience.

train_bank = X_train_bank.copy()
train_bank["Probability"] = y_train_bank

test_bank = X_test_bank_encoded.copy()
test_bank["Probability"] = y_test_bank

# Create textual representations. For instance, describe each data point:
def row_to_text(row):
    # Here we convert numeric/categorical features into a textual description.
    
    attributes = []
    for col in row.index:
        if col != "Probability":
            attributes.append(f"{col}={row[col]}")
    description = " ".join(attributes)
    return f"This individual has the following characteristics: {description}."

# Assuming you have train_bank and test_bank DataFrames
train_bank["text"] = train_bank.apply(row_to_text, axis=1)
test_bank["text"] = test_bank.apply(row_to_text, axis=1)

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_bank[["text", "Probability"]])
test_dataset = Dataset.from_pandas(test_bank[["text", "Probability"]])

In [9]:
# -------------------------------------
# 2. Tokenization
# -------------------------------------

model_name = "bert-base-uncased"  # You can choose another model if desired
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], padding="max_length", truncation=True, max_length=128
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Rename Probability column to labels
train_dataset = train_dataset.rename_column("Probability", "labels")
test_dataset = test_dataset.rename_column("Probability", "labels")

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

# Set the format for PyTorch
train_dataset = train_dataset.with_format("torch")
test_dataset = test_dataset.with_format("torch")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/7813 [00:00<?, ? examples/s]

Map:   0%|          | 0/3349 [00:00<?, ? examples/s]

In [10]:
# -------------------------------------
# 3. Load Pre-trained Model
# -------------------------------------

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# -------------------------------------
# 4. Define Metrics
# -------------------------------------

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1).numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [12]:
# -------------------------------------
# 5. Training Arguments and Trainer
# -------------------------------------

training_args = TrainingArguments(
    output_dir="WANDB-bank-SMOTE-50EP",
    evaluation_strategy="epoch",  # Ensure evaluation happens after each epoch
    save_strategy="epoch",        # Match save strategy to evaluation strategy
    save_total_limit=3,          # Keep only the latest checkpoint (final epoch)
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True  # Ensure the metric comparison works correctly
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_24/1382909829.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
# -------------------------------------
# 6. Train the Model
# -------------------------------------

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: arman-dogru. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241220_181047-dizo8r24
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run WANDB-bank-SMOTE-50EP
wandb: ⭐️ View project at https://wandb.ai/arman-dogru/huggingface
wandb: 🚀 View run at https://wandb.ai/arman-dogru/huggingface/runs/dizo8r24


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.493700,0.727144,0.589131,0.535995,0.989918,0.695440
2,0.360000,0.797612,0.553299,0.514845,0.994329,0.678418
3,0.340200,0.589421,0.683786,0.627536,0.818526,0.710418
4,0.328600,0.683411,0.621977,0.557838,0.975425,0.709766
5,0.334200,0.606173,0.679009,0.625613,0.803403,0.703448
6,0.276100,0.833113,0.631233,0.667939,0.441084,0.531309
7,0.221400,0.670787,0.641385,0.578201,0.899181,0.703822
8,0.272500,0.739555,0.662287,0.636364,0.670447,0.652961
9,0.308100,0.923549,0.663780,0.614506,0.779458,0.687222
10,0.299900,0.755602,0.647059,0.649667,0.553875,0.597959


TrainOutput(global_step=24450, training_loss=0.09590960735781814, metrics={'train_runtime': 5578.8869, 'train_samples_per_second': 70.023, 'train_steps_per_second': 4.383, 'total_flos': 2.5696083444096e+16, 'train_loss': 0.09590960735781814, 'epoch': 50.0})

In [15]:
# -------------------------------------
# 7. Evaluate the Model
# -------------------------------------

results = trainer.evaluate()
print("Evaluation results:", results)

Evaluation results: {'eval_loss': 3.0683934688568115, 'eval_accuracy': 0.6637802329053448, 'eval_precision': 0.5993106419646704, 'eval_recall': 0.8764965343415249, 'eval_f1': 0.7118730808597749, 'eval_runtime': 11.7205, 'eval_samples_per_second': 285.739, 'eval_steps_per_second': 17.917, 'epoch': 50.0}


In [16]:
# -------------------------------------
# 9. Save the Model
# -------------------------------------

model.save_pretrained("./fine_tuned_model_bank")
tokenizer.save_pretrained("./fine_tuned_model_bank")

('./fine_tuned_model_bank/tokenizer_config.json',
 './fine_tuned_model_bank/special_tokens_map.json',
 './fine_tuned_model_bank/vocab.txt',
 './fine_tuned_model_bank/added_tokens.json',
 './fine_tuned_model_bank/tokenizer.json')